In [2]:
import pandas as pd
import numpy as np
from scipy import stats

import src
from src import pandas_utils as pdu

from experiments.variant_density import parse_gnomad_variants

FILE_IN="data/interim/gnomad_snrna_variants_tidy_inbreeding_coeff.tsv"
FILE_OUT="data/final/gnomad_snrna_variants_hwe_stats.tsv"

In [3]:
def convert_to_float(df):
    cols = [
        "n_hom_ref",
        "n_het_alt",
        "n_hom_alt",
        "exp_hom_ref",
        "exp_het_alt",
        "exp_hom_alt",
    ]

    return df.astype({x: float for x in cols})


def anscome_correction(df):
    cols = [
        "n_hom_ref",
        "n_het_alt",
        "n_hom_alt",
        "exp_hom_ref",
        "exp_het_alt",
        "exp_hom_alt",
    ]
    mask0 = df.loc[:, cols].eq(0).any(axis=1)
    df.loc[mask0, cols] += 0.5

    return df


def hwe_stats(row):
    try: 
        chi2, p = stats.chisquare(
            f_obs=[row["n_hom_ref"], row["n_het_alt"], row["n_hom_alt"]],
            f_exp=[row["exp_hom_ref"], row["exp_het_alt"], row["exp_hom_alt"]],
            ddof=1,
        )

    except: 
        chi2, p = np.nan, np.nan

    return chi2, p


def hwe_stats_fisher(row):
    rng = np.random.default_rng()
    method = stats.PermutationMethod(n_resamples=100, rng=rng)

    print("Printing the ROW: ", row)

    res = stats.fisher_exact(
        [
            [row["n_hom_ref"], row["n_het_alt"], row["n_hom_alt"]],
            [row["exp_hom_ref"], row["exp_het_alt"], row["exp_hom_alt"]],
        ],
        method=method,
    )

    stat, p = res.statistic, res.pvalue
    
    # print(stat, p)
    return stat, p


df = (
    pd.read_csv(FILE_IN, sep="\t")
    .check.nrows(check_name="Input variants")
    .loc[lambda x: ~x["chrom"].isin(["chrX", "chrY"])]
    .check.nrows(check_name="Variants after dropping sex chromosomes")
    .check.value_counts("filter", check_name="Variant counts by filter:")
    .check.disable_checks(enable_asserts=False)
    .pipe(parse_gnomad_variants.parse_gnomad_variants)
    .check.enable_checks()
    .loc[
        :,
        [
            "chrom",
            "pos",
            "ref",
            "alt",
            "ac_nfe",
            "an_nfe",
            "af_nfe",
            "nhomalt_nfe",
            "allele_type",
            "ensg",
            "symbol",
            "gene_type",
        ],
    ]
    .loc[lambda x: x["ac_nfe"] > 0]
    .check.nrows(check_name="Variants with AC>0 in NFE")
    .rename(
        columns={
            "ac_nfe": "ac",
            "an_nfe": "an",
            "af_nfe": "af",
            "nhomalt_nfe": "n_hom_alt",
        }
    )
    .assign(
        ac_hom_alt=lambda x: x["n_hom_alt"] * 2,
        ac_het_alt=lambda x: x["ac"] - x["ac_hom_alt"],
        n_het_alt=lambda x: x["ac_het_alt"],
        n_total=lambda x: (x["an"] / 2).astype(int),
        n_hom_ref=lambda x: x["n_total"] - x["n_hom_alt"] - x["n_het_alt"],
        af_alt=lambda x: x["ac"] / x["an"],
        af_ref=lambda x: 1 - x["af_alt"],
        exp_hom_ref=lambda x: x["af_ref"] ** 2 * x["n_total"],
        exp_het_alt=lambda x: 2 * x["af_ref"] * x["af_alt"] * x["n_total"],
        exp_hom_alt=lambda x: x["af_alt"] ** 2 * x["n_total"],
    )
    .loc[
        :,
        [
            "chrom",
            "pos",
            "ref",
            "alt",
            "ac",
            "ac_het_alt",
            "ac_hom_alt",
            "an",
            "af",
            "af_ref",
            "af_alt",
            "allele_type",
            "ensg",
            "symbol",
            "gene_type",
            "n_total",
            "n_hom_ref",
            "n_het_alt",
            "n_hom_alt",
            "exp_hom_ref",
            "exp_het_alt",
            "exp_hom_alt",
        ],
    ]
    .pipe(convert_to_float)
    .pipe(lambda x: x.assign(**pd.DataFrame(x.apply(hwe_stats, axis=1).to_list(), index=x.index, columns=["chi2", "p"])))
    .sort_values("chi2", ascending=False)
    # .pipe(anscome_correction)
    # .pipe(pdu.assign_with_per_row_fn, hwe_stats_fisher, new_cols=["stat", "p"])
    # .sort_values("chi2", ascending=False)
    .check.head(10)
)

<h5 style='text-align: left'><span style='color:None; background-color:None'>Input variants: 29487</span></h5>

<h5 style='text-align: left'><span style='color:None; background-color:None'>Variants after dropping sex chromosomes: 28716</span></h5>

<h5 style='text-align: left'><span style='color:None; background-color:None'>Variant counts by filter:</span></h5>

,count
filter,
PASS,28708
InbreedingCoeff,8


<h5 style='text-align: left'><span style='color:None; background-color:None'>Variants with AC>0 in NFE: 14633</span></h5>

<h5 style='text-align: left'><span style='color:None; background-color:None'>First 10 rows</span></h5>

,chrom,pos,ref,alt,ac,ac_het_alt,ac_hom_alt,an,af,af_ref,af_alt,allele_type,ensg,symbol,gene_type,n_total,n_hom_ref,n_het_alt,n_hom_alt,exp_hom_ref,exp_het_alt,exp_hom_alt,chi2,p
5085,chr2,97664689,C,T,2,0,2,68030,0.000,1.000,0.000,SNV,ENSG00000201806.1,RNU4-8P,Pseudogene,34015,34014.000,0.000,1.000,34013.000,2.000,0.000,34015.000,0.000
1503,chr1,99978954,G,A,2,0,2,68026,0.000,1.000,0.000,SNV,ENSG00000212248.1,RNU6-750P,Pseudogene,34013,34012.000,0.000,1.000,34011.000,2.000,0.000,34013.000,0.000
652,chr1,31497626,C,T,2,0,2,68022,0.000,1.000,0.000,SNV,ENSG00000206981.1,RNU6-40P,Pseudogene,34011,34010.000,0.000,1.000,34009.000,2.000,0.000,34011.000,0.000
23068,chr14,99945450,A,C,2,0,2,68006,0.000,1.000,0.000,SNV,ENSG00000199836.1,RNU1-47P,Pseudogene,34003,34002.000,0.000,1.000,34001.000,2.000,0.000,34003.000,0.000
28360,chr22,10736274,G,T,33842,33842,0,67692,0.500,0.500,0.500,SNV,ENSG00000277248.1,U2,snRNA,33846,4.000,33842.000,0.000,8463.500,16923.000,8459.500,33830.004,0.000
13293,chr6,108292859,T,C,2,0,2,66968,0.000,1.000,0.000,SNV,ENSG00000206974.1,RNU6-1144P,Pseudogene,33484,33483.000,0.000,1.000,33482.000,2.000,0.000,33484.000,0.000
1829,chr1,143699510,C,T,2,0,2,66164,0.000,1.000,0.000,SNV,ENSG00000207349.1,RNVU1-17,snRNA,33082,33081.000,0.000,1.000,33080.000,2.000,0.000,33082.000,0.000
1821,chr1,143699467,G,A,2,0,2,66066,0.000,1.000,0.000,SNV,ENSG00000207349.1,RNVU1-17,snRNA,33033,33032.000,0.000,1.000,33031.000,2.000,0.000,33033.000,0.000
1867,chr1,143720564,C,T,2,0,2,66012,0.000,1.000,0.000,SNV,ENSG00000252826.1,RNVU1-23,snRNA,33006,33005.000,0.000,1.000,33004.000,2.000,0.000,33006.000,0.000
1852,chr1,143699607,G,A,2,0,2,65988,0.000,1.000,0.000,SNV,ENSG00000207349.1,RNVU1-17,snRNA,32994,32993.000,0.000,1.000,32992.000,2.000,0.000,32994.000,0.000


In [6]:
sub_df = df.loc[[1503, 652],:]

for i, row in sub_df.iterrows():
    print(row)
    print(hwe_stats(row))

chrom                       chr1
pos                     99978954
ref                            G
alt                            A
ac                             2
ac_het_alt                     0
ac_hom_alt                     2
an                         68026
af                      0.000029
af_ref                  0.999971
af_alt                  0.000029
allele_type                  SNV
ensg           ENSG00000212248.1
symbol                 RNU6-750P
gene_type             Pseudogene
n_total                    34013
n_hom_ref                34012.0
n_het_alt                    0.0
n_hom_alt                    1.0
exp_hom_ref         34011.000029
exp_het_alt             1.999941
exp_hom_alt             0.000029
chi2                     34013.0
p                            0.0
Name: 1503, dtype: object
(np.float64(34013.0), np.float64(0.0))
chrom                       chr1
pos                     31497626
ref                            C
alt                            T
ac         

In [8]:
stats.chisquare([34010.5,0.5, 1.5],[34009.500029, 2.499941,0.500029])

Power_divergenceResult(statistic=np.float64(3.599740777036422), pvalue=np.float64(0.16532031424392626))

In [11]:
stats.chisquare([1,5,10], [6,6,4])

Power_divergenceResult(statistic=np.float64(13.333333333333334), pvalue=np.float64(0.0012726338013398079))